In [80]:
# TensorFlow and tf.keras
import tensorflow as tf

# Helper libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

print(tf.__version__)

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

import sys
sys.path.append("../lib/")
from pathlib import Path

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score

2.12.0


In [81]:
dataset = "L_540_2022_C_E_R_B"
results_path = Path("../results/") / dataset
df = pd.read_csv(results_path / (dataset + "_preprocessed.csv"))


X = pd.get_dummies(df.drop(['label', 'file'], axis=1))

# Map the class names to integer values
class_map = {'Both': 0, 'Control': 1, 'Etoposide': 2, 'Resveratrol': 3}
y = df['label'] = df['label'].replace(class_map)

In [89]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

y_train

1143    3
1156    3
470     1
1113    3
394     1
       ..
646     2
9       0
1018    3
273     0
1146    3
Name: label, Length: 960, dtype: int64

In [4]:
class_names = ['Both', 'Control', 'Etoposide', 'Resveratrol']

# Build the model

In [90]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(2401,)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(4)
])


In [91]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Train the model

In [92]:
model.fit(X_train, y_train, epochs=100, use_multiprocessing = True)

Epoch 1/100
30/30 [==============================] - 1s 3ms/step - loss: 1.3310 - accuracy: 0.3677
Epoch 2/100
30/30 [==============================] - 0s 3ms/step - loss: 1.1903 - accuracy: 0.5198
Epoch 3/100
30/30 [==============================] - 0s 3ms/step - loss: 1.0819 - accuracy: 0.5583
Epoch 4/100
30/30 [==============================] - 0s 3ms/step - loss: 1.0233 - accuracy: 0.5792
Epoch 5/100
30/30 [==============================] - 0s 3ms/step - loss: 0.9598 - accuracy: 0.6083
Epoch 6/100
30/30 [==============================] - 0s 3ms/step - loss: 0.9313 - accuracy: 0.6198
Epoch 7/100
30/30 [==============================] - 0s 3ms/step - loss: 0.9039 - accuracy: 0.6365
Epoch 8/100
30/30 [==============================] - 0s 3ms/step - loss: 0.8680 - accuracy: 0.6458
Epoch 9/100
30/30 [==============================] - 0s 3ms/step - loss: 0.8526 - accuracy: 0.6521
Epoch 10/100
30/30 [==============================] - 0s 3ms/step - loss: 0.8312 - accuracy: 0.6479
Epoch 11/

In [94]:
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

print('\nTest accuracy:', test_acc)

8/8 - 0s - loss: 0.9669 - accuracy: 0.6375 - 46ms/epoch - 6ms/step

Test accuracy: 0.637499988079071


# Build the model

In [38]:
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])
predictions = probability_model.predict(X_test)
predictions[0]

8/8 [==============================] - 0s 2ms/step


array([4.2935785e-06, 9.5329696e-01, 2.3143338e-02, 2.3555385e-02],
      dtype=float32)

In [39]:
np.argmax(predictions[0])

1

# Use the trained model

In [60]:
# Grab an image from the test dataset.
test = X_test[:1]

# Tune model

In [ ]:
model.save('tfmodel')
del model 
model = load_model('tfmodel')